# Week 3: XGBoost Modeling and SHAP Interpretation

## Objectives
1. Load preprocessed data and baseline model
2. Train XGBoost classifier with class imbalance handling
3. Perform hyperparameter tuning
4. Evaluate and compare with baseline
5. Use SHAP for model interpretation
6. Generate comprehensive visualizations
7. Prepare final presentation materials

## Timeline
**Week 3 (Dec 2-8)** - Final Presentation: Dec 2

---

## Setup

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import joblib
from tqdm.auto import tqdm

# Machine learning
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score
)
import xgboost as xgb
import shap

# Settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# Random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ Libraries imported successfully")

In [ ]:
# Configuration
# Set this to True to skip hyperparameter tuning (faster, uses defaults)
# Set to False for full hyperparameter search (slower, better results)
SKIP_TUNING = False  # Change to True for quick testing

if SKIP_TUNING:
    print("⚠️  QUICK MODE: Skipping hyperparameter tuning")
    print("   Will use default parameters (faster but may not be optimal)")
else:
    print("✅ FULL MODE: Will perform hyperparameter tuning")
    print("   This may take 15-30 minutes depending on dataset size")

In [ ]:
# Set up paths
project_root = Path.cwd().parent
data_dir = project_root / 'data'
processed_data_dir = data_dir / 'processed'
models_dir = project_root / 'models'
figures_dir = project_root / 'figures'

print(f"Project root: {project_root}")
print(f"Models directory: {models_dir}")

## 1. Load Data and Baseline Model

In [ ]:
# Load dataset
data_file = processed_data_dir / 'hits_dataset.csv'

if not data_file.exists():
    raise FileNotFoundError(
        f"❌ Dataset not found. Please run notebooks 01 and 02 first!"
    )

df = pd.read_csv(data_file)
print(f"✅ Loaded dataset: {df.shape}")
print(f"\nClass distribution:")
print(df['is_hit'].value_counts())

In [ ]:
# Prepare features and target (same as Week 2)
exclude_cols = ['is_hit', 'year']
text_cols = df.select_dtypes(include=['object']).columns.tolist()
exclude_cols.extend(text_cols)

feature_cols = [col for col in df.columns if col not in exclude_cols]

X = df[feature_cols].values
y = df['is_hit'].values

print(f"Features: {feature_cols}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
# Split data (same split as Week 2 for fair comparison)
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_SEED,
    stratify=y
)

print(f"Train set: {len(X_train):,} samples")
print(f"Test set:  {len(X_test):,} samples")

In [ ]:
# Load baseline model for comparison
baseline_file = models_dir / 'baseline_logreg.pkl'
if baseline_file.exists():
    baseline_model = joblib.load(baseline_file)
    baseline_metrics = pd.read_csv(models_dir / 'baseline_metrics.csv')
    print("✅ Loaded baseline model")
    print("\nBaseline performance:")
    print(baseline_metrics.to_string(index=False))
else:
    print("⚠️  Baseline model not found. Run notebook 02 first for comparison.")
    baseline_model = None
    baseline_metrics = None

## 2. XGBoost with Default Parameters

First, let's train XGBoost with reasonable default parameters to establish a quick baseline.

In [ ]:
# Calculate scale_pos_weight for class imbalance
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count

print(f"Class imbalance handling:")
print(f"  Negative samples: {neg_count:,}")
print(f"  Positive samples: {pos_count:,}")
print(f"  scale_pos_weight: {scale_pos_weight:.2f}")
print(f"\n  This parameter tells XGBoost to give {scale_pos_weight:.1f}x more weight to hits.")

In [ ]:
# Train XGBoost with default parameters
print("Training XGBoost with default parameters...\n")

xgb_default = xgb.XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_SEED,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    eval_metric='logloss'
)

xgb_default.fit(X_train, y_train)

# Evaluate
y_pred_default = xgb_default.predict(X_test)
y_pred_proba_default = xgb_default.predict_proba(X_test)[:, 1]

print("✅ Default XGBoost trained")
print(f"\nQuick Performance Check:")
print(f"  Accuracy:  {accuracy_score(y_test, y_pred_default):.4f}")
print(f"  Precision: {precision_score(y_test, y_pred_default):.4f}")
print(f"  Recall:    {recall_score(y_test, y_pred_default):.4f}")
print(f"  F1 Score:  {f1_score(y_test, y_pred_default):.4f}")

## 3. Hyperparameter Tuning (Optional)

Use RandomizedSearchCV to find better hyperparameters.

In [ ]:
if not SKIP_TUNING:
    print("Starting hyperparameter tuning...")
    print("This may take 15-30 minutes.\n")
    
    # Define parameter grid
    param_distributions = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 4, 5, 6, 7, 8],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [1, 3, 5, 7],
        'gamma': [0, 0.1, 0.2, 0.3],
    }
    
    # Base model
    xgb_base = xgb.XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        random_state=RANDOM_SEED,
        eval_metric='logloss'
    )
    
    # Randomized search
    random_search = RandomizedSearchCV(
        xgb_base,
        param_distributions=param_distributions,
        n_iter=20,  # Number of parameter combinations to try
        cv=3,  # 3-fold cross-validation
        scoring='f1',  # Optimize for F1 score (good for imbalanced data)
        random_state=RANDOM_SEED,
        n_jobs=-1,  # Use all CPU cores
        verbose=1
    )
    
    # Fit
    random_search.fit(X_train, y_train)
    
    print("\n✅ Hyperparameter tuning complete!")
    print(f"\nBest parameters:")
    for param, value in random_search.best_params_.items():
        print(f"  {param}: {value}")
    print(f"\nBest cross-validation F1 score: {random_search.best_score_:.4f}")
    
    # Use best model
    xgb_final = random_search.best_estimator_
    
else:
    print("⏭️  Skipping hyperparameter tuning (using default parameters)")
    xgb_final = xgb_default

## 4. Final Model Training with Early Stopping

In [ ]:
# Train final model with early stopping to prevent overfitting
print("Training final XGBoost model with early stopping...\n")

# Split training data into train/validation
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=RANDOM_SEED,
    stratify=y_train
)

# Train with early stopping
xgb_final.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val, y_val)],
    verbose=False
)

print("✅ Final model trained")

## 5. Model Evaluation

In [ ]:
# Make predictions
y_pred = xgb_final.predict(X_test)
y_pred_proba = xgb_final.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

print("XGBOOST RESULTS")
print("="*60)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print(f"PR-AUC:    {pr_auc:.4f}")

# Compare with baseline
if baseline_metrics is not None:
    print("\n" + "="*60)
    print("COMPARISON WITH BASELINE")
    print("="*60)
    
    comparison = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC', 'PR-AUC'],
        'Logistic Regression': [
            baseline_metrics['accuracy'].values[0],
            baseline_metrics['precision'].values[0],
            baseline_metrics['recall'].values[0],
            baseline_metrics['f1_score'].values[0],
            baseline_metrics['roc_auc'].values[0],
            baseline_metrics['pr_auc'].values[0]
        ],
        'XGBoost': [accuracy, precision, recall, f1, roc_auc, pr_auc]
    })
    
    comparison['Improvement'] = comparison['XGBoost'] - comparison['Logistic Regression']
    comparison['Improvement %'] = (comparison['Improvement'] / comparison['Logistic Regression'] * 100).round(1)
    
    print(comparison.to_string(index=False))

### Confusion Matrix

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Non-Hit', 'Hit'],
            yticklabels=['Non-Hit', 'Hit'],
            cbar_kws={'label': 'Count'},
            ax=ax)

ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label', fontsize=12)
ax.set_title('Confusion Matrix - XGBoost', fontsize=14, fontweight='bold')

# Add percentage annotations
for i in range(2):
    for j in range(2):
        percentage = cm[i, j] / cm[i].sum() * 100
        ax.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)', 
                ha='center', va='center', fontsize=10, color='gray')

plt.tight_layout()
plt.savefig(figures_dir / 'xgboost_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: xgboost_confusion_matrix.png")

### Classification Report

In [ ]:
print("Classification Report:")
print("="*60)
print(classification_report(y_test, y_pred, 
                          target_names=['Non-Hit', 'Hit'],
                          digits=4))

## 6. Model Comparison Visualization

In [ ]:
if baseline_metrics is not None:
    # Side-by-side comparison
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Metrics comparison bar chart
    ax1 = axes[0]
    x = np.arange(len(comparison['Metric']))
    width = 0.35
    
    bars1 = ax1.bar(x - width/2, comparison['Logistic Regression'], width, 
                    label='Logistic Regression', color='#3498db')
    bars2 = ax1.bar(x + width/2, comparison['XGBoost'], width,
                    label='XGBoost', color='#2ecc71')
    
    ax1.set_xlabel('Metric', fontsize=12)
    ax1.set_ylabel('Score', fontsize=12)
    ax1.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(comparison['Metric'], rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.set_ylim([0, 1])
    
    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    # Improvement visualization
    ax2 = axes[1]
    colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in comparison['Improvement %']]
    bars = ax2.barh(comparison['Metric'], comparison['Improvement %'], color=colors)
    
    ax2.set_xlabel('Improvement (%)', fontsize=12)
    ax2.set_title('XGBoost Improvement over Baseline', fontsize=14, fontweight='bold')
    ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    ax2.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        label_x = width + (1 if width > 0 else -1)
        ax2.text(label_x, bar.get_y() + bar.get_height()/2.,
                f'{width:+.1f}%', ha='left' if width > 0 else 'right', 
                va='center', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(figures_dir / 'model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: model_comparison.png")

## 7. SHAP Analysis for Model Interpretation

SHAP (SHapley Additive exPlanations) helps us understand:
- Which features are most important
- How each feature affects predictions
- Individual prediction explanations

In [ ]:
print("Calculating SHAP values...")
print("This may take a few minutes.\n")

# Create SHAP explainer
# Use a sample for faster computation if dataset is large
sample_size = min(1000, len(X_test))
X_test_sample = X_test[:sample_size]
y_test_sample = y_test[:sample_size]

explainer = shap.TreeExplainer(xgb_final)
shap_values = explainer.shap_values(X_test_sample)

print(f"✅ SHAP values calculated for {sample_size} samples")

### SHAP Feature Importance

In [ ]:
# Feature importance based on mean absolute SHAP values
shap_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': np.abs(shap_values).mean(axis=0)
}).sort_values('Importance', ascending=False)

print("SHAP Feature Importance:")
print("="*60)
for idx, row in shap_importance.iterrows():
    print(f"{row['Feature']:20s} {row['Importance']:.4f}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(shap_importance)))
bars = ax.barh(range(len(shap_importance)), shap_importance['Importance'], color=colors)

ax.set_yticks(range(len(shap_importance)))
ax.set_yticklabels(shap_importance['Feature'])
ax.set_xlabel('Mean |SHAP Value|', fontsize=12)
ax.set_title('SHAP Feature Importance', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(figures_dir / 'shap_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: shap_feature_importance.png")

### SHAP Summary Plot

Shows both feature importance and effect direction.

In [ ]:
# Summary plot (beeswarm)
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test_sample, 
                  feature_names=feature_cols,
                  show=False)
plt.title('SHAP Summary Plot - Feature Effects', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(figures_dir / 'shap_summary_detailed.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: shap_summary_detailed.png")
print("\nHow to read this plot:")
print("  • Each dot is a song")
print("  • Red = high feature value, Blue = low feature value")
print("  • Right side (positive SHAP) = increases hit probability")
print("  • Left side (negative SHAP) = decreases hit probability")

### SHAP Dependence Plots

Show how individual features affect predictions.

In [ ]:
# Create dependence plots for top 4 features
top_features = shap_importance['Feature'].head(4).tolist()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, feature in enumerate(top_features):
    feature_idx = feature_cols.index(feature)
    shap.dependence_plot(
        feature_idx, 
        shap_values, 
        X_test_sample,
        feature_names=feature_cols,
        ax=axes[idx],
        show=False
    )
    axes[idx].set_title(f'SHAP Dependence: {feature}', fontweight='bold')

plt.suptitle('SHAP Dependence Plots - Top 4 Features', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(figures_dir / 'shap_dependence_plots.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: shap_dependence_plots.png")

### SHAP Force Plot (Sample Prediction Explanation)

In [ ]:
# Explain a single hit song prediction
hit_indices = np.where(y_test_sample == 1)[0]
if len(hit_indices) > 0:
    sample_idx = hit_indices[0]  # First hit in test set
    
    print(f"Explaining prediction for sample {sample_idx} (actual HIT)")
    print(f"Predicted probability: {xgb_final.predict_proba(X_test_sample[sample_idx:sample_idx+1])[:, 1][0]:.4f}")
    
    # Force plot
    shap.force_plot(
        explainer.expected_value,
        shap_values[sample_idx],
        X_test_sample[sample_idx],
        feature_names=feature_cols,
        matplotlib=True,
        show=False
    )
    plt.tight_layout()
    plt.savefig(figures_dir / 'shap_force_plot_example.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: shap_force_plot_example.png")

## 8. Key Insights from SHAP

In [ ]:
print("="*80)
print("KEY INSIGHTS FROM SHAP ANALYSIS")
print("="*80)

print(f"\n1. MOST IMPORTANT FEATURES FOR PREDICTING HITS:")
for i, (idx, row) in enumerate(shap_importance.head(5).iterrows(), 1):
    print(f"   {i}. {row['Feature']} (importance: {row['Importance']:.4f})")

print(f"\n2. FEATURE EFFECTS:")
for feature in top_features:
    feature_idx = feature_cols.index(feature)
    mean_shap = shap_values[:, feature_idx].mean()
    
    # Determine direction
    if mean_shap > 0:
        direction = "INCREASES"
        emoji = "📈"
    else:
        direction = "DECREASES"
        emoji = "📉"
    
    print(f"   {emoji} Higher {feature} generally {direction} hit probability")

print(f"\n3. MODEL INTERPRETATION:")
print(f"   • XGBoost captures non-linear relationships")
print(f"   • SHAP values show individual feature contributions")
print(f"   • Some features have complex interactions (see dependence plots)")

print(f"\n4. COMPARISON WITH LOGISTIC REGRESSION:")
if baseline_model is not None:
    # Compare feature importance
    lr_coef = pd.DataFrame({
        'Feature': feature_cols,
        'LR_Coefficient': baseline_model.coef_[0]
    })
    
    importance_comparison = shap_importance.merge(lr_coef, on='Feature')
    importance_comparison['LR_Abs_Coef'] = np.abs(importance_comparison['LR_Coefficient'])
    
    print(f"   Top features agree: ", end="")
    lr_top = set(lr_coef.nlargest(3, 'LR_Abs_Coef')['Feature'])
    shap_top = set(shap_importance.head(3)['Feature'])
    agreement = lr_top & shap_top
    print(f"{', '.join(agreement) if agreement else 'Different top features'}")
    
print("\n" + "="*80)

## 9. Save Final Model

In [ ]:
# Save XGBoost model
model_file = models_dir / 'final_xgboost.pkl'
joblib.dump(xgb_final, model_file)
print(f"✅ Saved model to: {model_file}")

# Save metrics
xgb_metrics = {
    'model': 'XGBoost',
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'roc_auc': roc_auc,
    'pr_auc': pr_auc
}

xgb_metrics_df = pd.DataFrame([xgb_metrics])
xgb_metrics_df.to_csv(models_dir / 'xgboost_metrics.csv', index=False)
print(f"✅ Saved metrics to: {models_dir / 'xgboost_metrics.csv'}")

# Save SHAP values for future use
np.save(models_dir / 'shap_values.npy', shap_values)
print(f"✅ Saved SHAP values to: {models_dir / 'shap_values.npy'}")

## 10. Final Summary Report

In [ ]:
print("="*80)
print("WEEK 3 FINAL SUMMARY REPORT")
print("="*80)

print(f"\n1. FINAL MODEL PERFORMANCE")
print(f"\n   XGBoost Metrics:")
print(f"     Accuracy:  {accuracy:.4f}")
print(f"     Precision: {precision:.4f}  ← {precision*100:.1f}% of predicted hits are correct")
print(f"     Recall:    {recall:.4f}  ← Catching {recall*100:.1f}% of actual hits")
print(f"     F1 Score:  {f1:.4f}  ← Balanced precision-recall")
print(f"     ROC-AUC:   {roc_auc:.4f}")
print(f"     PR-AUC:    {pr_auc:.4f}")

if baseline_metrics is not None:
    print(f"\n2. IMPROVEMENT OVER BASELINE")
    f1_improvement = ((f1 - baseline_metrics['f1_score'].values[0]) / 
                     baseline_metrics['f1_score'].values[0] * 100)
    recall_improvement = ((recall - baseline_metrics['recall'].values[0]) / 
                         baseline_metrics['recall'].values[0] * 100)
    
    print(f"     F1 Score: {f1_improvement:+.1f}% improvement")
    print(f"     Recall:   {recall_improvement:+.1f}% improvement")
    
    if f1_improvement > 5:
        print(f"     ✅ Significant improvement! XGBoost is superior.")
    elif f1_improvement > 0:
        print(f"     ✅ Modest improvement. XGBoost performs slightly better.")
    else:
        print(f"     ⚠️  Baseline was comparable. Consider feature engineering.")

print(f"\n3. TOP PREDICTIVE FEATURES (SHAP Analysis)")
for i, (idx, row) in enumerate(shap_importance.head(5).iterrows(), 1):
    feature = row['Feature']
    importance = row['Importance']
    
    # Get average SHAP value to determine direction
    feature_idx = feature_cols.index(feature)
    avg_shap = shap_values[:, feature_idx].mean()
    direction = "positive" if avg_shap > 0 else "negative"
    
    print(f"     {i}. {feature:20s} (importance: {importance:.4f}, {direction} effect)")

print(f"\n4. BUSINESS INSIGHTS")
print(f"\n   What Makes a Hit Song?")
print(f"   Based on our model analysis:")

# Analyze top features
for feature in shap_importance.head(3)['Feature']:
    feature_idx = feature_cols.index(feature)
    avg_shap = shap_values[:, feature_idx].mean()
    
    if avg_shap > 0:
        print(f"   • Higher {feature} → More likely to be a hit")
    else:
        print(f"   • Lower {feature} → More likely to be a hit")

print(f"\n5. MODEL LIMITATIONS")
print(f"   • Dataset limited to Spotify audio features")
print(f"   • Does not account for: marketing, artist fame, timing, luck")
print(f"   • Class imbalance ({(y==0).sum()/(y==1).sum():.1f}:1) makes prediction challenging")
print(f"   • Correlation ≠ causation (features associated with hits, not causing them)")

print(f"\n6. DELIVERABLES CREATED")
print(f"\n   Models:")
print(f"     • {models_dir / 'final_xgboost.pkl'}")
print(f"     • {models_dir / 'baseline_logreg.pkl'}")
print(f"     • {models_dir / 'scaler.pkl'}")

print(f"\n   Visualizations:")
print(f"     • {figures_dir / 'model_comparison.png'}")
print(f"     • {figures_dir / 'xgboost_confusion_matrix.png'}")
print(f"     • {figures_dir / 'shap_feature_importance.png'}")
print(f"     • {figures_dir / 'shap_summary_detailed.png'}")
print(f"     • {figures_dir / 'shap_dependence_plots.png'}")

print(f"\n   Data:")
print(f"     • {processed_data_dir / 'hits_dataset.csv'}")

print(f"\n7. PRESENTATION RECOMMENDATIONS")
print(f"\n   Key slides to include:")
print(f"   1. Problem: Predicting hit songs from audio features")
print(f"   2. Dataset: {len(df):,} songs, {(df['is_hit']==1).sum():,} hits ({(df['is_hit']==1).sum()/len(df)*100:.1f}%)")
print(f"   3. Methodology: Logistic Regression → XGBoost + SHAP")
print(f"   4. Results: Show model_comparison.png")
print(f"   5. Insights: Show shap_feature_importance.png")
print(f"   6. Deep dive: Show shap_summary_detailed.png")
print(f"   7. Limitations and future work")

print("\n" + "="*80)
print("✅ PROJECT COMPLETE - Ready for presentation!")
print("="*80)

---

## ✅ Week 3 Complete - Project Finished!

### What You've Accomplished:

1. ✅ **Comprehensive Data Analysis**
   - Loaded and cleaned Spotify dataset
   - Created HIT labels through dataset matching
   - Handled severe class imbalance

2. ✅ **Two-Model Approach**
   - Logistic Regression (interpretable baseline)
   - XGBoost (high-performance non-linear model)

3. ✅ **Advanced Interpretation**
   - SHAP values for feature importance
   - Dependence plots showing feature effects
   - Individual prediction explanations

4. ✅ **Professional Deliverables**
   - Publication-quality visualizations
   - Comprehensive evaluation metrics
   - Saved models for deployment

### For Your Presentation:

Use these figures:
- `model_comparison.png` - Show XGBoost improvements
- `shap_feature_importance.png` - Main "what makes a hit" slide
- `shap_summary_detailed.png` - Detailed feature effects
- `correlation_matrix.png` - Initial data exploration

### For Your Report:

Structure:
1. Introduction & Problem Statement
2. Dataset & Methodology
3. Exploratory Data Analysis (Week 1 figures)
4. Baseline Model (Week 2 results)
5. Advanced Model (Week 3 XGBoost)
6. SHAP Interpretation
7. Conclusions & Limitations

### Next Steps (If Time Permits):

- Try SMOTE for synthetic oversampling
- Feature engineering (interaction terms, temporal features)
- Try other models (Random Forest, Neural Networks)
- Build a simple web app for predictions

---

**Congratulations on completing this project! 🎉**

---